In [ ]:
import pandas as pd
import shutil
import os, pathlib
import cv2

In [ ]:
tar_dir = "/kaggle/working/train/"
if not os.path.exists(tar_dir):
    os.makedirs(tar_dir)
    
# define a function to count files in a directory
def move_files(path):
    count = 0
    for item in os.listdir(path):
        item_path = os.path.join(path, item)
        
        if os.path.isfile(item_path):
            tar_path = os.path.join(tar_dir + path.split('/')[-1])
            if not os.path.exists(tar_path):
                os.makedirs(tar_path)
            
            shutil.copy(item_path,tar_path)
        elif os.path.isdir(item_path):
            move_files(item_path)
    return 

# specify the root directory to traverse
root_dir = '/kaggle/input/data-pipeline-image-creation-128-x-256/train'

# call the count_files function to count the files
move_files(root_dir)


In [ ]:
def count_files(path):
    count = 0
    for item in os.listdir(path):
        item_path = os.path.join(path, item)
        if os.path.isfile(item_path):
            if item_path == '/kaggle/working/data-pipeline-image-creation-128-x-256/train/mabeat1/XC295336_0.jpeg':
               print("iii ", item_path)
            count += 1
        elif os.path.isdir(item_path):
            count += count_files(item_path)
    return count

# specify the root directory to traverse
root_dir = '/kaggle/working/'

# call the count_files function to count the files
num_files = count_files(root_dir)

# print the total number of files found
print(f'Total number of files: {num_files}')

In [ ]:
img_stats = pd.read_csv("/kaggle/input/data-pipeline-image-creation-128-x-256/img_stats.csv")
out_dir = "/kaggle/working/train/"
counts = img_stats['label'].value_counts().sort_values(ascending=False)
threshold = 100

In [ ]:
down_classes=[]

for idx,val in zip(counts.index, counts.values):
  if val<threshold:
    down_classes.append(idx)
    print(idx,val)

In [ ]:

for c in down_classes:
  class_df = img_stats.query("label==@c")
  print(class_df.shape[0])
  # # find number of samples to add
  num_up = threshold - class_df.shape[0]
  # # upsample the dataframe
  class_df_upsampled = class_df.sample(n=num_up, replace=True, random_state=40)
  #print("Later : ", class_df_upsampled.shape[0])
  # # append the upsampled dataframe to the list
  last_offset = class_df_upsampled['offset'].max()
  #print("Last offset :",last_offset)

  updated_df = []
  for index,row in class_df_upsampled.iterrows():
    last_offset+=1
    row['offset']=last_offset
    fname=row['filename'].split('.')[-2]
    fname= fname.split('_')[-2]
    fname = fname.split('/')[-1]
    fname = f"{fname}_{last_offset}.jpeg"
    rec_dir = out_dir + row["filename"].split('/')[0]
    src_img = out_dir + row["filename"]
    dest_img = os.path.join(rec_dir, fname)
    shutil.copy(src_img, dest_img)
    row.update({
        "offset":last_offset,
        "filename": "/".join(pathlib.Path(dest_img).parts[-2:]),
    })
    updated_df.append(row)
  updated_df = pd.DataFrame(updated_df)
  print(updated_df)
  img_stats = pd.concat([img_stats, updated_df],ignore_index=True)
     
img_stats.to_csv("img_stats_with_upsample.csv", index=False) 

In [ ]:
#shutil.move('/kaggle/input/data-pipeline-image-creation-128-x-256','/kaggle/working/')

In [3]:
# import shutil
# shutil.rmtree('/kaggle/working/')

OSError: [Errno 16] Device or resource busy: '/kaggle/working/'